In [ ]:
# Define the model dictionary initialization function
def initialize_best_estimators(grid_searches):
    return {
        'LR': LinearRegression(**grid_searches['LR'].best_params_),
        'RR': Ridge(**grid_searches['RR'].best_params_),
        'LAR': Lasso(**grid_searches['LAR'].best_params_),
        'ENR': ElasticNet(**grid_searches['ENR'].best_params_),
        'PLSR': PLSRegression(**grid_searches['PLSR'].best_params_),
        'SVR': SVR(**grid_searches['SVR'].best_params_)
    }

# Define a function to calculate the performance and characteristic coefficients of the computational model.
def evaluate_models(seed, X, Y, grid_searches, submodel_r2_sums, submodel_rmse_sums, submodel_intercepts, num_seeds):
    cross_validator = KFold(n_splits=2, shuffle=True, random_state=seed)
    best_estimators = initialize_best_estimators(grid_searches)

    # Create VotingRegressor
    submodels = [(name, estimator) for name, estimator in best_estimators.items() if estimator is not None]
    voting_regressor = VotingRegressor(submodels)

    # Calculate the mean values of R2 and RMSE for each submodel.
    submodel_r2_means = {}
    submodel_rmse_means = {}
    rmse_scorer = make_scorer(mean_squared_error, squared=False)
    
    # Used to store the feature coefficients of each model.
    submodel_feature_coefs = {}
    feature_coefs_weighted_sum = np.zeros(X.shape[1])
    total_r2 = 0

    for name, estimator in submodels:
        r2_scores = cross_val_score(estimator, X, Y, cv=cross_validator, scoring='r2', n_jobs=-1)
        rmse_scores = cross_val_score(estimator, X, Y, cv=cross_validator, scoring=rmse_scorer, n_jobs=-1)
        submodel_r2_means[name] = np.mean(r2_scores)
        submodel_rmse_means[name] = np.mean(rmse_scores)
        submodel_r2_sums[name] += submodel_r2_means[name]
        submodel_rmse_sums[name] += submodel_rmse_means[name]

        # Fit the model to obtain feature coefficients and intercepts.
        estimator.fit(X, Y)
        coefs = estimator.coef_.flatten()
        intercept = estimator.intercept_
        submodel_intercepts[name] += intercept / num_seeds
        
        submodel_feature_coefs[name] = dict(zip(X.columns, coefs))
        feature_coefs_weighted_sum += coefs * submodel_r2_means[name]
        total_r2 += submodel_r2_means[name]

    # Calculate the mean R2 and weighted RMSE of the fusion model.
    voting_regressor_r2_mean = np.mean(cross_val_score(voting_regressor, X, Y, cv=cross_validator, scoring='r2', n_jobs=-1))
    voting_regressor_rmse_mean = np.mean(cross_val_score(voting_regressor, X, Y, cv=cross_validator, scoring=rmse_scorer, n_jobs=-1))

    # Calculate weighted feature coefficients
    weighted_feature_coefs = feature_coefs_weighted_sum / total_r2 if total_r2 != 0 else feature_coefs_weighted_sum

    weighted_feature_coefs_dict = dict(zip(X.columns, weighted_feature_coefs))

    return {
        'submodel_r2_means': submodel_r2_means,
        'submodel_rmse_means': submodel_rmse_means,
        'voting_regressor_r2_mean': voting_regressor_r2_mean,
        'voting_regressor_rmse_mean': voting_regressor_rmse_mean,
        'submodel_feature_coefs': submodel_feature_coefs,
        'weighted_feature_coefs': weighted_feature_coefs_dict
    }

# Define and save the results of all seeds.
all_results = []
seeds = range(100)

# Initialize the sum of R2 and RMSE of submodels.
submodel_r2_sums = {name: 0.0 for name in initialize_best_estimators(grid_searches).keys()}
submodel_rmse_sums = {name: 0.0 for name in initialize_best_estimators(grid_searches).keys()}

# Initialize the sum of intercepts of submodels.
submodel_intercepts = {name: 0.0 for name in initialize_best_estimators(grid_searches).keys()}

# Initialize the weighted feature coefficient sum of the fusion model.
weighted_feature_coefs_sums = np.zeros(len(X.columns))

# Iterate through seeds and save results.
for seed in seeds:
    result = evaluate_models(seed, X, Y, grid_searches, submodel_r2_sums, submodel_rmse_sums, submodel_intercepts, len(seeds))
    all_results.append(result)
    weighted_feature_coefs_sums += np.array(list(result['weighted_feature_coefs'].values()))
    print(f"")
    print(f"Seed {seed} - Voting Regressor R2 Mean: {result['voting_regressor_r2_mean']}, RMSE Mean: {result['voting_regressor_rmse_mean']}")
    print("Submodel R2 and RMSE Means:")
    for name in result['submodel_r2_means'].keys():
        print(f"{name}: R2 Mean = {result['submodel_r2_means'][name]}, RMSE Mean = {result['submodel_rmse_means'][name]}")

# Calculate the average value of the weighted feature coefficients of the fusion model for all seeds.
avg_weighted_feature_coefs = weighted_feature_coefs_sums / len(seeds)

# Calculate the weighted average intercept of all seeds.
avg_weighted_intercept = sum(submodel_intercepts[name] for name in submodel_intercepts.keys()) / len(submodel_intercepts.keys())

# Bind the feature names and corresponding weighted feature coefficients, and sort them in descending order of importance.
sorted_feature_coefs = sorted(zip(X.columns, avg_weighted_feature_coefs), key=lambda x: x[1], reverse=True)

# Output the average R2, RMSE of the fusion model and the sorted feature coefficients.
avg_voting_regressor_r2_mean = np.mean([result['voting_regressor_r2_mean'] for result in all_results])
avg_voting_regressor_rmse_mean = np.mean([result['voting_regressor_rmse_mean'] for result in all_results])

print(f"")
print(f"Average Voting Regressor R2 Mean: {avg_voting_regressor_r2_mean}")
print(f"Average Voting Regressor Weighted RMSE Mean: {avg_voting_regressor_rmse_mean}")
print("Average Submodel R2 and RMSE Means:")
for name in submodel_r2_sums.keys():
    print(f"{name}: R2 Mean = {submodel_r2_sums[name] / len(seeds)}, RMSE Mean = {submodel_rmse_sums[name] / len(seeds)}")

# Draw a bar chart of feature coefficients.
features, coefs = zip(*sorted_feature_coefs)
plt.figure(figsize=(10, 8))
sns.barplot(x=list(coefs), y=list(features))
plt.xlabel('Feature Coefficients')
plt.ylabel('Feature')
plt.title('Feature Coefficients Sorted by Importance')
plt.show()

print(f"Average Weighted Intercept: {avg_weighted_intercept}")
print("Sorted Feature Coefficients:")
for feature, coef in sorted_feature_coefs:
    print(f"{feature}: {coef}")